### Bidirectional LSTM CNN for NER

Code adapted from: https://github.com/mxhofer/Named-Entity-Recognition-BidirectionalLSTM-CNN-CoNLL

In [2]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import TimeDistributed, Conv1D, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, MaxPooling1D, Flatten, concatenate
from keras.utils import plot_model
from keras.initializers import RandomUniform
from keras.optimizers import SGD, Nadam

In [ ]:
def read_file(file_name):
    """ return format 
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'] ...]
    """
    sentences = []
    sent = []
    f = open(file_name, 'r')
    for line in f:
        line = line.replace('\n', '')
        if !line.strip() or line.startswith('-DOCSTART'):
            continue
        else:
            words = line.split(' ')
            if words[0]=='.':
                sentences.append(sent)
                sent=[]
            else:
                sent.append([words[0], words[-1]])
    
    return sentences

read_file()

In [ ]:

# object mentioned to compatibility with 2.x
class CNN_BLSTM(object):
    
    def __init__(self, EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER):
        self.epochs = EPOCHS
        self.dropout = DROPOUT
        self.dropout_recurrent = DROPOUT_RECURRENT
        self.lstm_state_size = LSTM_STATE_SIZE
        self.conv_size = CONV_SIZE
        self.learning_rate = LEARNING_RATE
        self.optimizer = OPTIMIZER
        
    def load_data(self, file_locs):
        """Load data"""
        train_loc = file_locs['train']
        dev_loc = file_locs['dev']
        test_loc = file_locs['test']
        
        self.trans_sentences = read_file(train_loc)